In [345]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares
import matplotlib.pyplot as plt
from feos.eos import EquationOfState, PhaseDiagram
from feos.pcsaft import Identifier, IdentifierOption, PcSaftParameters, PcSaftRecord, PureRecord
from feos.eos.estimator import Estimator, Loss, DataSet, Phase
from feos.si import *

In [346]:
def eos_from_kij(kij):
    """Returns equation of state (PC-SAFT) for current kij."""
    global parameters
    p = PcSaftParameters.new_binary(parameters.pure_records, kij)
    return EquationOfState.pcsaft(p)

def cost(kij, estimator):
    """Calculates cost function for current parameters."""
    return estimator.cost(eos_from_kij(kij))

In [789]:
parameters = PcSaftParameters.from_json(["water", "1-butanol"], "gross2002.json")
pcsaft = EquationOfState.pcsaft(parameters)
parameters

|component|molarweight|$m$|$\sigma$|$\varepsilon$|$\mu$|$Q$|$\kappa_{AB}$|$\varepsilon_{AB}$|$N_A$|$N_B$|$N_C$|
|-|-|-|-|-|-|-|-|-|-|-|-|
|water|18.015|1.0656|3.0007|366.51|0|0|0.034868|2500.7|1|1|0|
|1-butanol|74.123|2.7515|3.6139|259.59|0|0|0.006692|2544.6|1|1|0|

In [ ]:
import pandas as pd
vle_bar = pd.read_excel("data_isotherm.xlsx", sheet_name = "water+1-butanol")
x_data = vle_bar.iloc[54:61,0].to_numpy()
t_data = (vle_bar.iloc[54:61,1] * 1.0).to_numpy()
y_data = vle_bar.iloc[54:61,2].to_numpy()
p_data = (vle_bar.iloc[54:61,3] / 1e5).to_numpy()


In [837]:
x_data

array([0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.562, 0.6, 0.7, 0.8, 0.9, 0.984],
      dtype=object)

In [838]:
df = pd.DataFrame({
    'x': x_data,
    't': t_data,
    'y': y_data,
    'p': p_data
})
df.to_csv('vle_isobar_t.csv',index = False)

In [839]:
data = pd.read_csv("vle_isobar_t.csv")

In [84]:
tmp = data.groupby('t')
print(tmp.describe())

              x                                                            \
          count    mean       std       min       25%       50%       75%   
t                                                                           
313.03004   9.0  0.7488  0.161763  0.373575  0.677394  0.772225  0.845705   

                        y            ...                         p            \
                max count      mean  ...       75%       max count      mean   
t                                    ...                                       
313.03004  0.887487   9.0  0.372572  ...  0.449344  0.540214   9.0  0.169601   

                                                                     
               std      min       25%       50%       75%       max  
t                                                                    
313.03004  0.04377  0.12559  0.142388  0.168519  0.195451  0.262512  

[1 rows x 24 columns]


In [ ]:
#isobaric condition
# isobarics = [
#     [
#         DataSet.binary_vle_pressure(
#             isobarc.t.values * KELVIN,
#             np.array([p] * len(isobarc)) * BAR,
#             isobarc.x.values,
#             Phase.Liquid
#         ),
#         DataSet.binary_vle_pressure(
#             isobarc.t.values * KELVIN,
#             np.array([p] * len(isobarc)) * BAR,
#             isobarc.y.values,
#             Phase.Vapor
#         ),

#     ]
#     for p, isobarc in data.groupby('p')
# ]

In [840]:
#isothermal condition
isotherms = [
    [
        DataSet.binary_vle_pressure(
            np.array([t] * len(isotherm)) * KELVIN,
            isotherm.p.values * BAR,
            isotherm.x.values,
            Phase.Liquid
        ),
        DataSet.binary_vle_pressure(
            np.array([t] * len(isotherm)) * KELVIN,
            isotherm.p.values * BAR,
            isotherm.y.values,
            Phase.Vapor
        ),

    ]
    for t, isotherm in data.groupby('t')
]

In [804]:
len(isotherms)

1

In [841]:
from itertools import chain
estimator_p = Estimator(list(chain.from_iterable(isotherms)), weights=[1]*2,losses = [Loss.linear()]*2)

In [ ]:
# estimator_p

In [842]:
initial_kij = [0.0] 
fitted_kij_p = least_squares(cost, initial_kij, bounds=[-0.5, 0.5], args=(estimator_p,),verbose=2).x

C:\Users\USER\AppData\Local\Temp\ipykernel_16176\2373019003.py:4: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  p = PcSaftParameters.new_binary(parameters.pure_records, kij)


   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.6817e-04                                    4.45e-03    
       1              2         1.0695e-04      6.12e-05       1.52e-02       9.55e-04    
       2              3         1.0510e-04      1.85e-06       2.23e-03       7.74e-05    
       3              4         1.0508e-04      1.44e-08       2.05e-04       9.34e-06    
       4              5         1.0508e-04      1.89e-10       2.34e-05       9.98e-07    
       5              6         1.0508e-04      2.40e-12       2.63e-06       1.18e-07    
       6              7         1.0508e-04      3.02e-14       2.95e-07       1.19e-08    
`ftol` termination condition is satisfied.
Function evaluations 7, initial cost 1.6817e-04, final cost 1.0508e-04, first-order optimality 1.19e-08.


In [843]:
fitted_kij_p

array([0.01312003])